In [1]:
import pandas as pd
from utils.plot_helper import plot_compare_predictions
from datetime import date
import os
from pathlib import Path

today = date.today()

tst_path = './results/data/2025-01-13_TSTransformer/'
nf_path = './results/data/2025-01-13_NFModels/'


In [2]:
op_length = 28
features = ['fatalities', 'event']

for feature in features:
    result_path = f'./results/images/compare/{today}/{feature}'
    os.makedirs(result_path, exist_ok=True)

    nf_data = pd.read_csv(f"{nf_path}/daily_{feature}_results.csv")
    nf_data.columns
    
    nf_df = nf_data[['ds', 'actual', 'DLinear', 'NBEATS', 'VanillaTransformer']].copy()
    directory = Path(f"{tst_path}/{feature}")
    file_count = len([file for file in directory.iterdir() if file.is_file()])

    for k in range(file_count):
        tst_data = pd.read_csv(f"{tst_path}/{feature}/{k:03d}.csv") 
        ds, actual, predict = tst_data.columns

        timestamps = tst_data[ds].values
        input_sequence = tst_data.loc[0:2*op_length-1, actual].values
        actual_sequence  = tst_data.loc[2*op_length:, actual].values

        tst = tst_data.loc[2*op_length:]

        nf_batch = nf_df[op_length*k:op_length*(k+1)].copy()
        nf_batch = nf_batch[nf_batch['ds'].isin(timestamps)].copy()
        predict_data = pd.merge(nf_batch, tst, left_on = 'ds', right_on = ds)
        predict_data = predict_data.drop(columns = ['ds', 'actual_x', 'actual_y', 'timestamps'])
        predict_data = predict_data.rename(columns = {predict:"TSTransformer"})
        if (k%op_length==0):
            plot_compare_predictions(input_sequence, actual_sequence, predict_data, timestamps, feature, result_path)



   
    